In [1]:
import cv2
import face_recognition
import numpy as np
import os
from datetime import datetime

path = 'students'
images = []
classNames = []
myList = os.listdir(path)

print(f'Found {len(myList)} files in {path} folder.')

for cl in myList:
    # Filter for image files only
    if cl.lower().endswith(('.png', '.jpg', '.jpeg')):
        curImg = cv2.imread(f'{path}/{cl}')
        images.append(curImg)
        classNames.append(os.path.splitext(cl)[0])

def findEncodings(images):
    encodeList = []
    for img in images:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        encode = face_recognition.face_encodings(img)[0]
        encodeList.append(encode)
    return encodeList

print('Encoding started... please wait.')
encodeListKnown = findEncodings(images)
print('Encoding Complete. Total faces encoded:', len(encodeListKnown))

# ==========================================
# STEP 2: ATTENDANCE LOGGING FUNCTION
# ==========================================
def markAttendance(name):
    # Ensure file exists or create it
    if not os.path.exists('Attendance.csv'):
        with open('Attendance.csv', 'w') as f:
            f.writelines('Name,Time,Date')
            
    with open('Attendance.csv', 'r+') as f:
        myDataList = f.readlines()
        nameList = [line.split(',')[0] for line in myDataList]
        
        if name not in nameList:
            now = datetime.now()
            timeString = now.strftime('%H:%M:%S')
            dateString = now.strftime('%d-%m-%Y')
            f.writelines(f'\n{name},{timeString},{dateString}')
            print(f"Attendance marked for: {name}")


cap = cv2.VideoCapture(0)

print("Starting Webcam... Point the camera at your face.")
print("Press 'q' to exit.")

while True:
    success, img = cap.read()
    if not success:
        print("Failed to grab frame")
        break

    # 1. Resize for performance (M1 can handle larger, but 0.25 is very smooth)
    imgS = cv2.resize(img, (0, 0), None, 0.25, 0.25)
    imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)

    # 2. Detect face locations and encodings
    # 'hog' is fast, 'cnn' is highly accurate but slower
    facesCurFrame = face_recognition.face_locations(imgS, model='hog')
    encodesCurFrame = face_recognition.face_encodings(imgS, facesCurFrame)

    # 3. Compare detected faces with known encodings
    for encodeFace, faceLoc in zip(encodesCurFrame, facesCurFrame):
        matches = face_recognition.compare_faces(encodeListKnown, encodeFace, tolerance=0.5)
        faceDis = face_recognition.face_distance(encodeListKnown, encodeFace)
        
        if len(faceDis) > 0:
            matchIndex = np.argmin(faceDis)

            if matches[matchIndex]:
                name = classNames[matchIndex].upper()
                
                # 4. Draw bounding box and label
                # Since we resized to 1/4th, we must multiply coordinates by 4
                y1, x2, y2, x1 = faceLoc
                y1, x2, y2, x1 = y1*4, x2*4, y2*4, x1*4
                
                cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.rectangle(img, (x1, y2-35), (x2, y2), (0, 255, 0), cv2.FILLED)
                cv2.putText(img, name, (x1+6, y2-6), cv2.FONT_HERSHEY_COMPLEX, 0.8, (255, 255, 255), 2)
                
                # 5. Log the entry
                markAttendance(name)

    # Display the result
    cv2.imshow('M1 Attendance Monitor', img)
    
    # Check for 'q' key to stop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


cap.release()
cv2.destroyAllWindows()
# Mac-specific fix for frozen windows
for i in range(5):
    cv2.waitKey(1)

/Users/arif/miniforge3/envs/tf/lib/python3.10/site-packages/face_recognition_models/__init__.py:7: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_filename


FileNotFoundError: [Errno 2] No such file or directory: 'students'